<a href="https://colab.research.google.com/github/maidacundo/test_referees_optimization/blob/main/test_ortools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install ortools

In [2]:
# test di uno use case semplice + minization di un objective:

# - 15 arbitri (0..8)
# - 3 giorni (0..2)
# - 6 squadre (0..5)

# - 3 arbitri per partita

# --------------------------------------------

# arbitro 0 non è disponibile il giorno 0
# arbitro 1 non è disponibile il giorno 1
# arbitro 2 non è disponibile il giorno 2

# arbitro 1 non deve arbitrare il match 8 (2, 4)

# arbitro 0 ha arbitrato la squadra 0 già 5 volte in passato
# arbitro 1 ha arbitrato la squadra 0 già 5 volte in passato

# arbitri è meglio se arbitrano tutti lo stesso numero di partite
# arbitri è meglio se arbitrano le squadre in maniera omogenea

#modellazione con or-tools (cp-sat model)
from ortools.sat.python import cp_model

model = cp_model.CpModel()

squadre = [0, 1, 2, 3, 4, 5]

calendar = [
    [(0, 1), (2, 3), (4, 5)],
    [(0, 2), (1, 4), (3, 5)],
    [(0, 3), (1, 5), (2, 4)],
    [(0, 1), (2, 3), (4, 5)],
    [(0, 2), (1, 4), (3, 5)],
    [(0, 3), (1, 5), (2, 4)],
    [(0, 1), (2, 3), (4, 5)],
    [(0, 2), (1, 4), (3, 5)],
    [(0, 3), (1, 5), (2, 4)]]

num_referee = 40
num_days = len(calendar)
num_teams = 6
min_referee_per_match = 3


In [3]:
# lista per tenere traccia di quante volte un arbitro ha arbitrato una partita in passato

referee_history = []
for i in range(num_referee):
  referee_history.append([])
  for s in range(len(squadre)):
    referee_history[i].append(0)

referee_history[0][0] = 1
referee_history[1][1] = 1
referee_history[2][2] = 1
referee_history[3][3] = 1
referee_history[4][4] = 1
referee_history[5][5] = 1
referee_history[6][0] = 1

referee_history[0][1] = 1
referee_history[1][2] = 1
referee_history[2][3] = 1
referee_history[3][4] = 1
referee_history[4][5] = 1
referee_history[5][0] = 1
referee_history[6][1] = 1

referee_history

[[1, 1, 0, 0, 0, 0],
 [0, 1, 1, 0, 0, 0],
 [0, 0, 1, 1, 0, 0],
 [0, 0, 0, 1, 1, 0],
 [0, 0, 0, 0, 1, 1],
 [1, 0, 0, 0, 0, 1],
 [1, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0]]

In [4]:
x = []
for i in range(num_referee):
    r = []
    for day in range(num_days):
        num_matches = len(calendar[day])
        d = []
        for j in range(num_matches):
            d.append(model.NewIntVar(0, 1, 'referee{}_day{}_match{}'.format(i, day, j)))
        r.append(d)
    x.append(r)

In [5]:
# definisco le constraints del modello 

# Ogni arbitro deve arbitrare almeno una volta
for i in range(num_referee):
    model.AddAtLeastOne(x[i][d][j] for d in range(num_days) for j in range(num_matches))

# Ogni arbitro è assegnato al max ad una partita per giorno
for i in range(num_referee):
    for d in range(num_days):
        model.AddAtMostOne(x[i][d][j] for j in range(num_matches))

# Ogni partita ha 3 arbitri
for d in range(num_days):
    for j in range(num_matches):
        model.Add(sum([x[i][d][j] for i in range(num_referee)]) == min_referee_per_match)
    
# Arbitro 0 non è disponibile il giorno 0
referee_not_available = 0
day_not_available = 0
num_matches = len(calendar[day_not_available])
model.Add(sum(x[referee_not_available][day_not_available][j] for j in range(num_matches)) == 0)

# Arbitro 1 non è disponibile il giorno 1
referee_not_available = 1
day_not_available = 1
num_matches = len(calendar[day_not_available])
model.Add(sum(x[referee_not_available][day_not_available][j] for j in range(num_matches)) == 0)

# Arbitro 2 non è disponibile il giorno 2
referee_not_available = 2
day_not_available = 2
num_matches = len(calendar[day_not_available])
model.Add(sum(x[referee_not_available][day_not_available][j] for j in range(num_matches)) == 0)

# Arbitro 8 non è disponibile il giorno 3
referee_not_available = 8
day_not_available = 3
num_matches = len(calendar[day_not_available])
model.Add(sum(x[referee_not_available][day_not_available][j] for j in range(num_matches)) == 0)

# Arbitro 1 non deve arbitrare il match 8
referee_not_available = 1
day_of_match = 2
match_not_available = 2
model.Add(x[referee_not_available][day_of_match][match_not_available] == 0)

In [6]:
calendar

[[(0, 1), (2, 3), (4, 5)],
 [(0, 2), (1, 4), (3, 5)],
 [(0, 3), (1, 5), (2, 4)],
 [(0, 1), (2, 3), (4, 5)],
 [(0, 2), (1, 4), (3, 5)],
 [(0, 3), (1, 5), (2, 4)],
 [(0, 1), (2, 3), (4, 5)],
 [(0, 2), (1, 4), (3, 5)],
 [(0, 3), (1, 5), (2, 4)]]

In [7]:
# calcolo delle squadre arbitrate da ogni singolo arbitro

referee_matches_per_team = []

for i in range(num_referee):
  teams_per_referee = []
  for s in squadre:
    matches_per_team = []
    for d in range(num_days):
      for m in calendar[d]:
        if s in m:
          j = calendar[d].index(m)
          matches_per_team.append(x[i][d][j])
    
    try:
      previous_matches_number = referee_history[i][s]
    except IndexError:
      previous_matches_number = 0

    # commentare per utilizzare lo storico delle partite
    previous_matches_number = 0
    
    teams_per_referee.append(sum(matches_per_team) + previous_matches_number)
  referee_matches_per_team.append(teams_per_referee)

# referee_matches_per_team[referee][team] ritorna la somma di partite arbitrate da quell'arbitro per quella squadra
referee_matches_per_team[1][0]

Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(referee1_day0_match0(0..1), referee1_day1_match0(0..1)), referee1_day2_match0(0..1)), referee1_day3_match0(0..1)), referee1_day4_match0(0..1)), referee1_day5_match0(0..1)), referee1_day6_match0(0..1)), referee1_day7_match0(0..1)), referee1_day8_match0(0..1))

In [8]:
team_diff = []
all_variables = []

for i in range(len(referee_matches_per_team)):
  t = []
  for s in squadre:
    c = []
    for j in range(len(referee_matches_per_team)):
      if i != j:
        diff_var = model.NewIntVar(0, num_days, f'matches_diff_{i}_{j}_team_{s}') # todo controllare meglio up e lb!
        abs_diff = model.AddAbsEquality(diff_var, (referee_matches_per_team[i][s] - referee_matches_per_team[j][s]))
        c.append(diff_var)
        all_variables.append(diff_var)
    
    t.append(sum(c))
  team_diff.append(sum(t))

# aggiungo una decision stategy (euristica) per migliorare le performance
model.AddDecisionStrategy(all_variables, cp_model.CHOOSE_LOWEST_MIN, cp_model.SELECT_MIN_VALUE)

# v[referee] ritorna la differenza totate di quell'arbitro per tutti i team rispetto a tutti gli altri arbitri

team_diff_per_referre_to_minimize = sum(team_diff)
team_diff_per_referre_to_minimize

Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(Sum(matches_diff_0_1_team_0(0..9), matches_diff_0_2_team_0(0..9)), matches_diff_0_3_team_0(0..9)), matches_diff_0_4_team_0(0..9)), matches_diff_0_5_team_0(0..9)), matches_diff_0_6_team_0(0..9)), matches_diff_0_7_team_0(0..9)), matches_diff_0_8_team_0(0..9)), matches_diff_0_9_team_0(0..9)), matches_diff_0_10_team_0(0..9)), matches_diff_0_11_team_0(0..9)), matches_diff_0_12_team_0(0..9)), matches_diff_0_13_team_0(0..9)), matches_diff_0_14_team_0(0..9)), matches_diff_0_15_team_0(0..9)), matches_diff_0_16_team_0(0..9)), matches_diff_0_17_team_0(0..9)), matches_diff_0_18_team_0(0..9)), matches_diff_0_19_team_0(0..9)), matches_diff_0_20_team_0(0..9)), matches_diff_0_21_team

In [9]:
# calcolo dei match totali arbitrati da ogni singolo arbitro

total_matches_per_referee = []

for i in range(num_referee):
    total_matches = 0
    for d in range(num_days):
        total_matches += sum(x[i][d])
    total_matches_per_referee.append(total_matches)

In [10]:
matches_differences = []
all_variables = []

for i in range(len(total_matches_per_referee)):
    c = []
    for j in range(len(total_matches_per_referee)):
        if i != j:
            diff_var = model.NewIntVar(0, num_days, f'matches_diff_{i}_{j}')
            abs_diff = model.AddAbsEquality(diff_var, (total_matches_per_referee[i] - total_matches_per_referee[j]))
            c.append(diff_var)
            all_variables.append(diff_var)
    matches_differences.append(c)

# aggiungo una decision stategy (euristica) per migliorare le performance
model.AddDecisionStrategy(all_variables, cp_model.CHOOSE_LOWEST_MIN, cp_model.SELECT_LOWER_HALF)

# matches_differences[referee] ritorna la lista di differenze tra quell'arbitro e gli altri

In [11]:
matches_differences[0]

[matches_diff_0_1(0..9),
 matches_diff_0_2(0..9),
 matches_diff_0_3(0..9),
 matches_diff_0_4(0..9),
 matches_diff_0_5(0..9),
 matches_diff_0_6(0..9),
 matches_diff_0_7(0..9),
 matches_diff_0_8(0..9),
 matches_diff_0_9(0..9),
 matches_diff_0_10(0..9),
 matches_diff_0_11(0..9),
 matches_diff_0_12(0..9),
 matches_diff_0_13(0..9),
 matches_diff_0_14(0..9),
 matches_diff_0_15(0..9),
 matches_diff_0_16(0..9),
 matches_diff_0_17(0..9),
 matches_diff_0_18(0..9),
 matches_diff_0_19(0..9),
 matches_diff_0_20(0..9),
 matches_diff_0_21(0..9),
 matches_diff_0_22(0..9),
 matches_diff_0_23(0..9),
 matches_diff_0_24(0..9),
 matches_diff_0_25(0..9),
 matches_diff_0_26(0..9),
 matches_diff_0_27(0..9),
 matches_diff_0_28(0..9),
 matches_diff_0_29(0..9),
 matches_diff_0_30(0..9),
 matches_diff_0_31(0..9),
 matches_diff_0_32(0..9),
 matches_diff_0_33(0..9),
 matches_diff_0_34(0..9),
 matches_diff_0_35(0..9),
 matches_diff_0_36(0..9),
 matches_diff_0_37(0..9),
 matches_diff_0_38(0..9),
 matches_diff_0_39(0.

In [12]:
total_differences_per_referee = []
for i in range(len(matches_differences)):
    total_differences_per_referee.append(sum(matches_differences[i]))
  
total_diff_per_referre_to_minimize = sum(total_differences_per_referee)

In [13]:
# lista contenente tutti i valori da minimizzare

values_to_minimize = []
values_to_minimize.append(team_diff_per_referre_to_minimize * 0.7)
values_to_minimize.append(total_diff_per_referre_to_minimize * 0.3)

In [14]:
model.Minimize(sum(values_to_minimize))
# model.Minimize(team_diff_per_referre_to_minimize)

In [15]:
print(model.ModelStats())

optimization model '':
Search strategy: on 9360 variables, CHOOSE_LOWEST_MIN, SELECT_MIN_VALUE
Search strategy: on 1560 variables, CHOOSE_LOWEST_MIN, SELECT_LOWER_HALF
#Variables: 12000 ( in floating point objective)
  - 1080 in [0,1]
  - 10920 in [0,9]
#kAtMostOne: 360 (#literals: 1080)
#kBoolOr: 40 (#literals: 1080)
#kLinMax: 10920
#kLinear1: 1
#kLinear3: 4
#kLinearN: 27 (#terms: 1080)


In [16]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables, print_boolean = False):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0
        self.print_boolean = print_boolean

    def on_solution_callback(self):
        self.__solution_count += 1
        
        print(self.__solution_count)
        print(f'Differenza totale per partite arbitrate: {self.Value(total_diff_per_referre_to_minimize)}')
        print(f'Differenza totale per squadre arbitrate: {self.Value(team_diff_per_referre_to_minimize)}')
        print('-'* 30)

        if self.print_boolean:
          print('POSSIBILE SOLUZIONE')
          match_per_referee = [0] * num_referee

          for d in range(num_days):
              print('-' * 30)
              print(f'Day {d}:\n')
              num_matches = len(calendar[d])
              for i in range(num_referee):
                  for j in range(num_matches):
                      if self.BooleanValue(self.__variables[i][d][j]):
                          match_per_referee[i] += 1
                          print(
                              f'Referee {i} assigned to match {j}')
          print()
          print('TOTALI PARTITE PER ARBITRO')
          for i in range(num_referee):
            print(f'Referee {i}: {match_per_referee[i]}')
          print()

    def solution_count(self):
        return self.__solution_count


In [17]:
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter(x)

#solver.parameters.enumerate_all_solutions = True
solver.parameters.num_search_workers = 4

solver.parameters.max_time_in_seconds = 600.0

status = solver.Solve(model, solution_printer)

1
Differenza totale per partite arbitrate: 1310
Differenza totale per squadre arbitrate: 4420
------------------------------
2
Differenza totale per partite arbitrate: 1238
Differenza totale per squadre arbitrate: 4212
------------------------------


In [20]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('POSSIBILE SOLUZIONE')

    match_per_referee = [0] * num_referee

    for d in range(num_days):
        print('-' * 30)
        print(f'Day {d}:\n')
        num_matches = len(calendar[d])
        for i in range(num_referee):
            for j in range(num_matches):
                if solver.BooleanValue(x[i][d][j]):
                    match_per_referee[i] += 1
                    print(
                        f'Referee {i} assigned to match {calendar[d][j]}')
    print()
    print('TOTALI PARTITE PER ARBITRO')
    for i in range(num_referee):
      print(f'Referee {i}: {match_per_referee[i]}')
    
    print(f'Differenza totale per partite arbitrate: {solver.Value(total_diff_per_referre_to_minimize)}')
    print(f'Differenza totale per squadre arbitrate: {solver.Value(team_diff_per_referre_to_minimize)}')

    print()
    print('TOTALI PARTITE PER SQUADRA')
    for i in range(num_referee):
      print(f'Referee {i}: {solver.Value(team_diff[i])}')

else:
    print('No solution found.')

POSSIBILE SOLUZIONE
------------------------------
Day 0:

Referee 2 assigned to match (0, 1)
Referee 14 assigned to match (0, 1)
Referee 20 assigned to match (2, 3)
Referee 23 assigned to match (4, 5)
Referee 24 assigned to match (4, 5)
Referee 32 assigned to match (2, 3)
Referee 33 assigned to match (4, 5)
Referee 34 assigned to match (2, 3)
Referee 39 assigned to match (0, 1)
------------------------------
Day 1:

Referee 0 assigned to match (0, 2)
Referee 2 assigned to match (3, 5)
Referee 6 assigned to match (0, 2)
Referee 10 assigned to match (1, 4)
Referee 11 assigned to match (0, 2)
Referee 21 assigned to match (1, 4)
Referee 26 assigned to match (1, 4)
Referee 27 assigned to match (3, 5)
Referee 28 assigned to match (3, 5)
------------------------------
Day 2:

Referee 5 assigned to match (2, 4)
Referee 7 assigned to match (0, 3)
Referee 9 assigned to match (2, 4)
Referee 13 assigned to match (0, 3)
Referee 15 assigned to match (1, 5)
Referee 16 assigned to match (2, 4)
Refere

In [19]:
# - 15 arbitri (0..8)
# - 3 giorni (0..2)
# - 6 squadre (0..5)

# - 3 arbitri per partita

# --------------------------------------------

# arbitro 0 non è disponibile il giorno 0
# arbitro 1 non è disponibile il giorno 1
# arbitro 2 non è disponibile il giorno 2

# arbitro 1 non deve arbitrare il match 8 (2, 4)

# arbitro 0 ha arbitrato la squadra 0 già 5 volte in passato
# arbitro 1 ha arbitrato la squadra 0 già 5 volte in passato